##### Student Name: Dara Leonard
##### Student ID: 19202478

In [7]:
import pandas as pd
from stravalib import Client

strava_client = Client(access_token="26f827483092d875bc5986834686d28fad87a38a")

res = strava_client.explore_segments(bounds=[53.263329, -9.070642, 53.283310, -9.044810], activity_type='running') # Get current athlete details
seg_info = [strava_client.get_segment(seg.id) for seg in res]

columns = ["segment_id", "segment_name", "distance", "best_time", "best_time_date", "location", "city", "effort_count"]
df = pd.DataFrame(columns=columns)
for i, seg in enumerate(seg_info):
    segment_leaderboard = strava_client.get_segment_leaderboard(seg.id)
    tmp = dict({"segment_id": seg.id,
             "segment_name": seg.name,
             "distance": seg.distance,
             "best_time" : segment_leaderboard.entries[0].elapsed_time,
             "best_time_date" : segment_leaderboard.entries[0].start_date_local,
             "location": seg.state,
             "city": seg.city,
             "effort_count": seg.effort_count})
    df.loc[i] = tmp
    
df

,segment_id,segment_name,distance,best_time,best_time_date,location,city,effort_count
0,4878158,Quincentennial Bridge (West to East),975.30 m,00:01:33,2019-12-04 20:18:05,Galway,Galway,2210
1,11123727,town loop,4125.70 m,00:18:01,2016-02-25 19:07:26,Limerick,None,37
2,11517987,Aughinish long trail,3532.30 m,00:14:21,2018-04-13 18:58:26,Limerick,None,509
3,6617028,Unnamed Road Climb,1819.70 m,00:15:49,2016-09-11 08:42:31,Clare,"Clare, Co. Clare, Ireland",13
